In [9]:
import numpy as np
from datetime import datetime
import lyricsgenius
import json
import time

In [2]:
genius_client_access_token = 'fB4ldLe2jlKTIM7rNpHEFDozmXC-ipojy2wYZPS3rICuruFr7TH0Q9eED3CkewS8'
genius = lyricsgenius.Genius(genius_client_access_token)

In [3]:
example = [
    {
        'name': None,
        'id': None,
        'albums': [
            {
                'name':None,
                'id': None,
                'tracks': []
            }
        ]
    },
    

]

In [4]:
def dropDictKeys(dict, keys):
    for key in keys:
        try:
            del dict[key]
        except Exception as e:
            pass
    return dict

def keepDictKeys(dict, keys):
    new_dict = {}
    for key in keys:
        new_dict[key] = dict[key]
    return new_dict

In [12]:
print(f"Starting: {datetime.now()}")
artist_name = 'Drake'

artist = genius.search_artist(artist_name, 0)
artist_id = json.loads(artist.to_json())['id']

artist_albums = genius.artist_albums(artist_id)['albums']

artist = artist_albums[0]['artist']
drop_keys = ['_type', 'api_path', 'index_character', 'is_meme_verified', 'slug']
artist = dropDictKeys(artist, drop_keys)

artist['albums'] = []
for j, album in enumerate(artist_albums):
    album = dropDictKeys(album, ['artist'])
    print(f"\n {j}: {album['name']} \n")

    if album['release_date_components']:
        date_comps = album['release_date_components']
        date_string = f"{date_comps['year']}-{date_comps['month']}-{date_comps['day']}".replace('None', '01')
        if 'None' not in date_string.split('-'):
            album['date'] = datetime.strptime(date_string, '%Y-%m-%d').date()
        else:
            album['date'] = np.nan
    
    drop_cols = ['_type', 'api_path', 'release_date_components']
    album = dropDictKeys(album, drop_cols)

    album['tracks'] = []
    artist['albums'].append(album)

    max_albums = 50
    album_id = album['id']
    album_tracks = genius.album_tracks(album_id, max_albums)['tracks']

    for i, track in enumerate(album_tracks):

        track_data = {}
        track_data['number'] = track['number']

        for k, v in track['song'].items():
            track_data[k] = v 

        print(f"{i}: {track_data['title']}")

        drop_cols = ['_type', 'annotation_count', 'api_path', 'instrumental', 'lyrics_owner_id', 'lyrics_updated_at', 'path', 'pyongs_count', 'updated_by_human_at', 'primary_artist']

        track_data = dropDictKeys(track_data, drop_cols)
        album['tracks'].append(track_data)

        track_id = track_data['id']

        
        try:
            song = genius.song(track_id)['song']
            song['lyrics'] = genius.lyrics(track_id)
        except Exception:
            time.sleep(1)
            song = song
        

        keep_cols = ['lyrics', 'description', 'embed_content', 'featured_video', 'recording_location', 'apple_music_id', 'apple_music_player_url', 'release_date', 'release_date_for_display', 'featured_artists', 'producer_artists', 'writer_artists', 'custom_performances', 'song_relationships']
        song = keepDictKeys(song, keep_cols)
        for k, v in song.items():
            track_data[k] = v
print(f"Starting: {datetime.now()}")

Starting: 2021-01-05 18:54:29.162096
Searching for songs by Drake...

Done. Found 0 songs.

 0: Certified Lover Boy 

0: Lucky Lefty
1: Laugh Now Cry Later
2: Intoxicated*
3: Need Me (Lotta 42)
4: I Did (Grand Entrance)*
5: Not Around
6: In the Cut
7: Lie To Me
8: Certified Lover Boy*
9: Like I’m Supposed To/Do Things
10: Zodiac Sign

 1: Dark Lane Demo Tapes 

0: Deep Pockets
1: When to Say When
2: Chicago Freestyle
3: Not You Too
4: Toosie Slide
5: Desires
6: Time Flies
7: Landed
8: D4L
9: Pain 1993
10: Losses
11: From Florida With Love
12: Demons
13: War

 2: Care Package 

0: Dreams Money Can Buy
1: The Motion
2: How Bout Now
3: Trust Issues
4: Days in the East
5: Draft Day
6: 4PM in Calabasas
7: 5 AM in Toronto
8: I Get Lonely
9: My Side
10: Jodeci Freestyle
11: Club Paradise
12: Free Spirit
13: Heat of the Moment
14: Girls Love Beyoncé
15: Paris Morton Music
16: Can I

 3: The Best in the World Pack 

0: Omertà
1: Money in the Grave

 4: Scorpion 

0: Survival
1: Nonstop
2: Eleva

In [17]:
1 + 1

2

In [18]:
import pickle
pickle.dump(artist, open('artist.pkl', 'wb'))